(10 points) (Exercise 7.9) For the prostate data of Chapter 3, carry out a bestsubset linear regression analysis,
as in Table 3.3 (third column from the left). Compute the AIC, BIC, five- and tenfold cross-validation, and bootstrap .632 estimates of prediction error.

In [54]:
import pandas as pd
import numpy as np
import statsmodels
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture # AIC BIC
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.linear_model import LassoLarsIC
from sklearn.model_selection import KFold  # or just kfold with n 
from sklearn.neighbors import KNeighborsClassifier


from mlxtend.feature_selection import ExhaustiveFeatureSelector as BSS # includes forward and backwawrds selection

In [73]:
df = pd.read_csv('./PROSTATE_DATA/prostate.csv', delimiter="\t") 

In [49]:
# INPUT PROSTATE DATA
prostate_data = df.set_index('Unnamed: 0') 
del prostate_data.index.name
print(prostate_data.head())

     lcavol   lweight  age      lbph  svi       lcp  gleason  pgg45      lpsa  \
1 -0.579818  2.769459   50 -1.386294    0 -1.386294        6      0 -0.430783   
2 -0.994252  3.319626   58 -1.386294    0 -1.386294        6      0 -0.162519   
3 -0.510826  2.691243   74 -1.386294    0 -1.386294        7     20 -0.162519   
4 -1.203973  3.282789   58 -1.386294    0 -1.386294        6      0 -0.162519   
5  0.751416  3.432373   62 -1.386294    0 -1.386294        6      0  0.371564   

  train  
1     T  
2     T  
3     T  
4     T  
5     T  


In [50]:
prostate_data_target = prostate_data.train
prostate_data_predictors = prostate_data.drop('train', axis = 1)  
# print(prostate_data_target.tolist())

In [51]:
# print(len(prostate_data_predictors))
true_false_list = []
prostate_list = prostate_data_target.tolist()
for i in range(0 , len(prostate_data_target)): 
    if prostate_list[i] == "T":
        true_false_list.append(True)
    else :
        true_false_list.append(False)
true_false_list = pd.DataFrame({"Target" : true_false_list})
prostate_data_target = true_false_list
print(prostate_data_target)

    Target
0     True
1     True
2     True
3     True
4     True
..     ...
92    True
93    True
94   False
95    True
96   False

[97 rows x 1 columns]


In [52]:
x_train,x_test,y_train,y_test=train_test_split(prostate_data_predictors,prostate_data_target,test_size=0.2)

In [70]:
# lm = LinearRegression()
# lm = LogisticRegression()
lm = LassoLarsIC()
# knn = KNeighborsClassifier(n_neighbors=3)

bss = BSS(lm, 
           min_features=1,
           max_features=4,
           scoring='accuracy',
           print_progress=True,
           cv=5) 

bss = bss.fit(prostate_data_predictors, prostate_data_target)


/home/mavericku/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [3]:
def best_subset(X, y, k_features = [3, 4]):
    n_features = X.shape[1]
    subsets = chain.from_iterable(combinations(xrange(n_features), k+1) for k in k_features)
    best_score = -np.inf
    best_subset = None
    for subset in subsets:
        lin_reg = sm.OLS(y, X.iloc[:, subset]).fit()
        pred = lin_reg.predict()
        score = lin_reg.rsquared_adj
        # print score
        if score > best_score:
            best_score, best_subset, best_reg_summary = score, f, reg.summary()
    return best_subset, best_score

In [65]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

iris = load_iris()
X = iris.data
y = iris.target
print(y)

knn = KNeighborsClassifier(n_neighbors=3)

efs1 = EFS(knn, 
           min_features=1,
           max_features=4,
           scoring='accuracy',
           print_progress=True,
           cv=5)

efs1 = efs1.fit(X, y)

print('Best accuracy score: %.2f' % efs1.best_score_)
print('Best subset (indices):', efs1.best_idx_)
print('Best subset (corresponding names):', efs1.best_feature_names_)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Features: 15/15

Best accuracy score: 0.97
Best subset (indices): (0, 2, 3)
Best subset (corresponding names): ('0', '2', '3')
